In [3]:
### import necessary libraries
import os
import ee
import sys
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt

# Sube un nivel en la estructura de directorios para llegar a la raíz del proyecto
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Ahora puedes importar la función refined_lee
from utils.refined_lee import refined_lee

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [4]:

# Define dates
before_start = '2018-07-15'
before_end = '2018-08-10'
after_start = '2018-08-10'
after_end = '2018-08-23'

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
ernakulam = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Ernakulam'))
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
geometry = ernakulam.geometry()
Map = geemap.Map()
# Load the Ernakulam district geometry
Map.addLayer(geometry, {'color': 'grey'}, 'Ernakulam District')

# Filter the image collection
collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.eq('resolution_meters', 10))\
    .filter(ee.Filter.bounds(geometry))\
    .select('VH')

# Filter collections for before and after the specified dates
before_collection = collection.filter(ee.Filter.date(before_start, before_end))
after_collection = collection.filter(ee.Filter.date(after_start, after_end))

# Mosaic and clip images
before = before_collection.mosaic().clip(geometry)
after = after_collection.mosaic().clip(geometry)

# Center map and add layers
Map.centerObject(geometry, 10)
Map.addLayer(before, {'min': -25, 'max': 0}, 'Before Floods', False)
Map.addLayer(after, {'min': -25, 'max': 0}, 'After Floods', False)

# Refined Lee Filter (Kernel)

# Speckle Filtering Functions
def to_natural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

def to_db(img):
    return ee.Image(img).log10().multiply(10.0)

before_filtered = ee.Image(to_db(refined_lee(to_natural(before))))
after_filtered = ee.Image(to_db(refined_lee(to_natural(after))))

Map.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Floods Filtered', False)
Map.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Floods Filtered', False)

# Exercise
# A simple method for filtering speckles is using a focal median filter
# Apply a Focal Median filter on both before and after images
# Use a Circle kernel with a 30 meter radius
# Add the filtered images to the map
# Hint: Use the foal_median() function

# Apply Focal Median filter on before image
before_filtered = before.focal_median(radius=30, units='meters')

# Apply Focal Median filter on after image
after_filtered = after.focal_median(radius=30, units='meters')

# Add filtered images to the map
Map.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Floods Filtered', False)
Map.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Floods Filtered', False)


difference = after_filtered.divide(before_filtered)
diff_threshold = 1.25
# Initial Estimate
flooded = difference.gt(diff_threshold).selfMask()
# Add layers to the map
Map.addLayer(flooded, {'palette': 'orange'}, 'Initial Flooded Areas')


In [21]:
# Lets apply masks

# Mask Permanent Water
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
permanent_water = gsw.select('seasonality').gte(5).clip(geometry)

# Mask - Pixel 1 will be retained, pizel 0 will be removed
permanent_water_mask = permanent_water.unmask(0).Not()

flooded = flooded.updateMask(permanent_water_mask)

#Add to map
Map.addLayer(flooded, {min:0,max:1,'palette': 'red'}, 'Permanent Water')

# Mask out areas with more than 5 percent slope using the HydroSHEDS DEM
dem = ee.Image('WWF/HydroSHEDS/03VFDEM')
slope = ee.Terrain.slope(dem).clip(geometry)
slope_mask = slope.lt(5)



In [12]:
Map

Map(center=[10.055341788485276, 76.46953395211857], controls=(WidgetControl(options=['position', 'transparent_…